In [1]:
import py_d3
py_d3.load_ipython_extension(get_ipython())

In [3]:
%%d3
<div></div>
<script>
var q = encodeURIComponent(get_sparql()),
    endpoint = "http://api.hojin-info.go.jp/sparql";

function get_sparql(){
    return [
        "PREFIX hj: <http://hojin-info.go.jp/ns/domain/biz/1#>",
        "PREFIX ic: <http://imi.go.jp/ns/core/rdf#>",
        "SELECT ?v ?name FROM <http://hojin-info.go.jp/graph/hojin>",
        "WHERE {",
        "?d hj:法人基本情報 ?db .",
        "?db ic:ID/ic:識別値 ?v ;",
        "ic:名称/ic:表記 ?name .",
        "}"
    ].join(" ");
}

d3.json(endpoint + "?query=" + q , function(error, data){
    console.log(endpoint + '?query=' + q)
    if(error){
        console.log(error)
    }else{
        show(data)
    }
});

function show(data){
    items = data['results']['bindings']
    d3.select('div').append("text").text(items.length)
}

</script>